# **Identificación de Imágenes Auténticas y Sintéticas : Abordando los Desafíos de las Imágenes Sintéticas en la Sociedad Actual**

**Contenido**
1. [Configuración del Entorno](#title1)
2. [Construcción del Conjunto de Datos](#title2)
3. [Creación del Modelo](#title3)

## Configuración del Entorno<a name="title1"></a>

In [1]:
import tensorflow as tf
print(tf.config.experimental.list_physical_devices("GPU"))
print(tf.__version__)
# Para limitar la memoria que usa la GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar TensorFlow para que solo utilice la GPU si es necesario
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Configuración de GPU completada")
    except RuntimeError as e:
        print(e)
# Verifica que la GPU está disponible
print("¿GPU está disponible?:", tf.config.list_physical_devices('GPU'))
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import gc
import tensorflow as tf
from keras import backend as K

# Limpiar la memoria del backend
K.clear_session()
# Forzar la recolección de basura
gc.collect()


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.0
Configuración de GPU completada
¿GPU está disponible?: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


0

In [ ]:
# Limpiar la memoria del backend
K.clear_session()
# Forzar la recolección de basura
gc.collect()

In [2]:
#@title Importar todas las librerías de uso.

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import glob
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, Flatten, MaxPooling2D, Dropout, SpatialDropout2D
from tensorflow.keras.metrics import Accuracy, AUC, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn import metrics
import matplotlib.image as mpmig
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

sns.set_theme()

## Construcción del Conjunto de Datos y Preprocesandolos<a name="title2"></a>

Fuente: https://www.kaggle.com/datasets/kaustubhdhote/human-faces-dataset & https://www.kaggle.com/datasets/hamzaboulahia/hardfakevsrealfaces/code

In [3]:
# Función para cargar imágenes por lotes
def load_images_in_batches(input_folder, target_size=(178, 178), batch_size=100):
    images = []  # Lista para almacenar imágenes
    labels = []  # Lista para almacenar etiquetas (basadas en nombres de carpetas)

    for subfolder in os.listdir(input_folder):
        subfolder_path = os.path.join(input_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for i, img_name in enumerate(os.listdir(subfolder_path)):
                if img_name.endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)  # Leer la imagen directamente sin conversión
                    resized_img = cv2.resize(img, target_size)
                    images.append(resized_img.astype('float32') / 255.0)  # Normalización y cambio a float32
                    labels.append(subfolder)

                    # Procesar en lotes
                    if len(images) >= batch_size:
                        yield np.array(images), np.array(labels)
                        images, labels = [], []  # Reiniciar las listas para el siguiente lote

    if images:  # Enviar las imágenes restantes
        yield np.array(images), np.array(labels)

# Cargar imágenes de entrenamiento
train_folder = './Data/Train'
train_images = []
train_labels = []

for images, labels in load_images_in_batches(train_folder, target_size=(178, 178), batch_size=100):
    train_images.extend(images)
    train_labels.extend(labels)

# Convertir a numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Preprocesamiento de etiquetas para clasificación binaria (real=0, fake=1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train_labels)
y_categorical = to_categorical(y_encoded)  # Para clasificación binaria, tendrá dos clases [0, 1]

# Cargar imágenes de testeo
test_folder = './Data/Test'
test_images = []
test_labels = []

for images, labels in load_images_in_batches(test_folder, target_size=(178, 178), batch_size=100):
    test_images.extend(images)
    test_labels.extend(labels)

test_images = np.array(test_images)
test_labels = np.array(test_labels)
y_test_encoded = label_encoder.transform(test_labels)
y_test_categorical = to_categorical(y_test_encoded)

# Modelo<a name="title3"></a>



In [4]:
#@title Construcción del modelo
model = models.Sequential([
    layers.Input(shape=(178, 178, 3)),

    # Bloque convolucional 1
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),

    # Bloque convolucional 2
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Bloque convolucional 3
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    # Aplanamiento y capas densas
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Salida para clasificación binaria
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 176, 176, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 174, 174, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 87, 87, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 87, 87, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 85, 85, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 83, 83, 128)       1

In [5]:
history = model.fit(train_images, y_categorical, batch_size=8, epochs=10, validation_data=(test_images, y_test_categorical))

Epoch 1/10
1204/1204 [==============================] - 40s 30ms/step - loss: 0.0709 - accuracy: 0.9780 - val_loss: 0.2896 - val_accuracy: 0.8813
Epoch 2/10
1204/1204 [==============================] - 35s 29ms/step - loss: 0.0499 - accuracy: 0.9848 - val_loss: 0.9104 - val_accuracy: 0.6625
Epoch 3/10
1204/1204 [==============================] - 34s 28ms/step - loss: 0.0302 - accuracy: 0.9919 - val_loss: 0.1647 - val_accuracy: 0.9426
Epoch 4/10
1204/1204 [==============================] - 34s 28ms/step - loss: 0.0370 - accuracy: 0.9889 - val_loss: 0.5300 - val_accuracy: 0.8177
Epoch 5/10
1204/1204 [==============================] - 34s 28ms/step - loss: 0.0402 - accuracy: 0.9881 - val_loss: 0.2922 - val_accuracy: 0.9077
Epoch 6/10
1204/1204 [==============================] - 34s 28ms/step - loss: 0.0411 - accuracy: 0.9882 - val_loss: 0.4991 - val_accuracy: 0.8223
Epoch 7/10
1204/1204 [==============================] - 34s 28ms/step - loss: 0.0165 - accuracy: 0.9948 - val_loss: 0.1825 -

In [7]:
#@title Metricas

# Evaluación del modelo
test_loss, test_accuracy = model.evaluate(test_images, y_test_categorical)
print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

# Predicciones
predictions = model.predict(test_images)
predicted_labels = (predictions > 0.5).astype(int).reshape(-1)  # Convertir probabilidades a etiquetas (0 o 1)
true_labels = y_test_encoded  # Etiquetas verdaderas en formato entero

# Matriz de confusión
conf_matrix = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Etiqueta Predicha')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión')
plt.show()

# Reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(true_labels, predicted_labels))

# Precisión global
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Precisión Global: {accuracy}")

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/conv2d_1/Relu' defined at (most recent call last):
    File "c:\Users\isidr\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\isidr\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\isidr\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\isidr\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\isidr\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\isidr\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asynci

In [ ]:
# Visualizar los resultados de entrenamiento
plt.plot(history.history['accuracy'], label='Precisión entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Pérdida entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()